In [1]:
import pandas as pd
flights = pd.read_csv('flights.csv')
airports = pd.read_csv('airports.csv')
airlines = pd.read_csv('airlines.csv')

c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
#Using count will output the number of non-missing values for each column in the dataset..Thus, telling us how many values are actually missing in each column 
flights.count()

#List of columns having missing values 
list_nacols = flights.columns[flights.isna().any()].tolist()
list_nacols

#For each column in the above list, printing their values too..
val_nacols = flights.loc[:, list_nacols]
val_nacols

#Replacing NaN values with 0 
flights = flights.fillna(0)
flights.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,0,0.0,0.0,0.0,0.0,0.0
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,0,0.0,0.0,0.0,0.0,0.0
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,0,0.0,0.0,0.0,0.0,0.0
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,0,0.0,0.0,0.0,0.0,0.0
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,0,0.0,0.0,0.0,0.0,0.0


In [3]:
#Combining year,month and day to create a timestamp
flights['DATE'] = pd.to_datetime(flights[['YEAR','MONTH','DAY']])
flights1 = flights
flights1= flights1.drop('YEAR',axis=1)
flights1 = flights1.drop('MONTH',axis =1)
flights1 = flights1.drop('DAY',axis= 1)


MemoryError: Unable to allocate 710. MiB for an array with shape (16, 5819079) and data type float64

In [ ]:
#Rearranging the columns of flights1 dataframe
flights1 = flights1[['DATE','DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER', 'TAIL_NUMBER',
       'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE',
       'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT', 'WHEELS_OFF',
       'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'WHEELS_ON',
       'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'ARRIVAL_DELAY',
       'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON', 'AIR_SYSTEM_DELAY',
       'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY',
       'WEATHER_DELAY']]


In [ ]:
flights2 = flights1[['DATE', 'AIRLINE', 'FLIGHT_NUMBER','ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE',
          'DEPARTURE_TIME', 'DEPARTURE_DELAY','SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'ARRIVAL_DELAY']]

flights2.head()

In [ ]:
print("Shape of flights2 dataframe:", flights2.shape)

In [ ]:
#December
flights_dec = flights2[flights2['DATE'] >= '2015-12-01']
flights_dec = flights_dec.reset_index()
flights_dec.head()


In [ ]:
print("Shape of flights_dec dataframe:", flights_dec.shape)

In [ ]:
import datetime
import numpy as np
def hour_format(x):
    if pd.isnull(x):
        return np.nan
    else:
        if x == 2400: x = 0
        x = "{0:04d}".format(int(x))
        hour = datetime.time(int(x[0:2]), int(x[2:4]))
        return hour

# Function that combines a date and time to produce a datetime.datetime
def combine_date_with_hour(x):
    if pd.isnull(x[0]) or pd.isnull(x[1]):
        return np.nan
    else:
        return datetime.datetime.combine(x[0],x[1])

# Function that combine two columns of the dataframe to create a datetime format
def flight_time(df, col):    
    list1 = []
    for index, cols in df[['DATE', col]].iterrows(): 
        if pd.isnull(cols[1]):
            list1.append(np.nan)
        elif float(cols[1]) == 2400:
            cols[0] += datetime.timedelta(days=1)
            cols[1] = datetime.time(0,0)
            add = combine_date_with_hour(cols)
            list1.append(add)
        else:
            cols[1] = hour_format(cols[1])
            add = combine_date_with_hour(cols)
            list1.append(combine_date_with_hour(cols))
    return pd.Series(list1)


In [ ]:
flights_dec['SCHEDULED_DEPARTURE'] = flight_time(flights_dec, 'SCHEDULED_DEPARTURE')
flights_dec['DEPARTURE_TIME'] = flights_dec['DEPARTURE_TIME'].apply(hour_format)
flights_dec['SCHEDULED_ARRIVAL'] = flights_dec['SCHEDULED_ARRIVAL'].apply(hour_format)
flights_dec['ARRIVAL_TIME'] = flights_dec['ARRIVAL_TIME'].apply(hour_format)


In [ ]:
flights_dec.loc[:5, ['SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'SCHEDULED_ARRIVAL',
            'ARRIVAL_TIME', 'ARRIVAL_DELAY']]

In [ ]:
names_airlines = airlines.set_index('IATA_CODE')['AIRLINE'].to_dict()
names_airlines

flights_dec['AIRLINE_NAMES'] = flights_dec['AIRLINE'].map(names_airlines)
flights_dec = flights_dec[['DATE','AIRLINE','AIRLINE_NAMES','FLIGHT_NUMBER','ORIGIN_AIRPORT','DESTINATION_AIRPORT','SCHEDULED_DEPARTURE','DEPARTURE_TIME','DEPARTURE_DELAY','SCHEDULED_ARRIVAL','ARRIVAL_TIME','ARRIVAL_DELAY']]
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns


fig = plt.figure(1, figsize=(10,5))
ax = sns.barplot(x='AIRLINE_NAMES', y="DEPARTURE_DELAY", data=flights_dec, color="lightblue", ci=None)
ax = sns.barplot(x="AIRLINE_NAMES", y="ARRIVAL_DELAY", data=flights_dec,  hatch = '--',alpha = 0.0, ci=None)
plt.ylabel('Departure_delay & Arrival_delay',fontsize=10, labelpad=5)
plt.xticks(rotation=90)

ax.legend(["Departure_Delay", "Arrival_Delay"],bbox_to_anchor=(1, 1))

In [ ]:
flights_dec.head()
from sklearn import preprocessing
x=flights_dec.drop('DATE', 1)
x=x.drop('AIRLINE', 1)
x=x.drop('AIRLINE_NAMES', 1)
x=x.drop('SCHEDULED_DEPARTURE', 1)
x=x.drop('DEPARTURE_TIME', 1)

x=x.drop('DEPARTURE_DELAY', 1)
x=x.drop('SCHEDULED_ARRIVAL', 1)
x=x.drop('ARRIVAL_TIME', 1)
x=x.drop('ARRIVAL_DELAY', 1)
x.head()
label_encoder = preprocessing.LabelEncoder()
 
# Encode labels in column 'species'.
x['ORIGIN_AIRPORT']= label_encoder.fit_transform(x['ORIGIN_AIRPORT'])
label_encoder1 = preprocessing.LabelEncoder()
x['DESTINATION_AIRPORT']= label_encoder.fit_transform(x['DESTINATION_AIRPORT'])
x.head()

In [ ]:
y=flights_dec.ARRIVAL_DELAY
from sklearn import preprocessing
 
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
 
# Encode labels in column 'species'.
y= label_encoder.fit_transform(y)

In [ ]:
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report
X_train, X_test, y_train, y_test=train_test_split(x,y,test_size=0.1, random_state=42)
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
clf = SVC()
clf.fit(X_train,y_train)


pred=clf.predict(X_test)
print(classification_report(pred, y_test))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators = 100) 
 
# Training the model on the training dataset
# fit function is used to train the model using the training sets as parameters
clf.fit(X_train, y_train)
pred=clf.predict(X_test)
print(classification_report(pred, y_test))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

#Create KNN Classifier
clf = KNeighborsClassifier(n_neighbors=5)

#Train the model using the training sets
clf.fit(X_train, y_train)
pred=clf.predict(X_test)
print(classification_report(pred, y_test))

In [ ]:
import pymysql as mdb
import time

def upd(res,ids):
    
    mydb = mdb.connect(
      host="127.0.0.1",
      user="root",
      passwd="",
      database="tms"
    )

    mycursor = mydb.cursor()

   

    sql = "UPDATE sens SET delay = %s WHERE id = %s"
    val = (res,ids)

    mycursor.execute(sql, val)

    mydb.commit()
    mydb.close()
while True:
    time.sleep(2)
    mydb = mdb.connect(
          host="127.0.0.1",
          user="root",
          passwd="",
          database="tms"
        )

    mycursor = mydb.cursor()

    sql = "SELECT a,b,c,d,id FROM sens"


    mycursor.execute(sql)
   
    myresult = mycursor.fetchall()
    mydb.close()
    for x in myresult:            
        a=str(x[0])
        b=str(x[1])
        c=str(x[2])
        res=str(x[3])
        ids=str(x[4])
        
        if(res==''):
            
            aa=[]
            comm=str(comm)
            aa.append(a)
            aa.append(b)
            aa.append(c)
            
            pred=clf.predict(aa)
            res=str(pred[0])
            upd(res,ids)
        
